## Welch's Method for Power Spectral Density Estimation

Welch's method is a technique for estimating the power spectral density of a signal. It's an improvement over the basic periodogram method as it reduces noise in the PSD estimate by averaging multiple modified periodograms.

---

### 1. Windowing Function

A **window function** is applied to each segment of the signal to reduce spectral leakage. Common window functions include Hamming, Hann, and Rectangular.
The code defines $w[n]$ based on the `window_type` parameter.

* **Hamming Window:**
    $$w[n] = 0.54 - 0.46 \cos\left(\frac{2\pi n}{M-1}\right), \quad 0 \le n \le M-1$$
* **Hann Window:**
    $$w[n] = 0.5 - 0.5 \cos\left(\frac{2\pi n}{M-1}\right), \quad 0 \le n \le M-1$$
* **Rectangular Window:**
    $$w[n] = 1, \quad 0 \le n \le M-1$$

Where $M$ is the `segment_length`.

---

### 2. Normalization Factor ($U$)

The **normalization factor** $U$ accounts for the power of the window function. It ensures that the PSD estimate has the correct units.

$$U = \sum_{n=0}^{M-1} w[n]^2$$

---

### 3. Segmenting the Signal

The input signal is divided into $K$ overlapping segments. The overlap percentage is defined by `overlap`.

* **Number of segments ($K$):**
    $$K = \left\lfloor \frac{N - M}{\text{step}} \right\rfloor + 1$$
    where $N$ is the total length of the signal, $M$ is the `segment_length`, and `step` is given by:
    $$\text{step} = M \times (1 - \text{overlap})$$

* **Individual Segment ($s_k[n]$):**
    Each segment $k$ is extracted and multiplied by the window function:
    $$s_k[n] = x[n + k \cdot \text{step}] \cdot w[n], \quad 0 \le n \le M-1$$
    where $x[n]$ is the original signal.

---

### 4. Discrete Fourier Transform (DFT) of Each Segment

For each windowed segment $s_k[n]$, its **Discrete Fourier Transform (DFT)** is computed. This is done using the Fast Fourier Transform (FFT) algorithm.

$$X_k[f] = \sum_{n=0}^{M-1} s_k[n] \cdot e^{-j 2\pi f n / M}$$

---

### 5. Periodogram of Each Segment ($P_k[f]$)

The **periodogram** for each segment is calculated from the magnitude squared of its DFT, normalized by the sampling frequency ($f_s$) and the window normalization factor ($U$).

$$P_k[f] = \frac{1}{f_s \cdot U} |X_k[f]|^2$$

---

### 6. Averaging the Periodograms

The final Welch PSD estimate is obtained by **averaging the periodograms** of all $K$ segments.

$$P_{\text{welch}}[f] = \frac{1}{K} \sum_{k=0}^{K-1} P_k[f]$$

---

### 7. Frequency Vector ($f$)

The corresponding **frequency values** for the PSD are generated using:

$$f = \text{FFTShift}\left(\text{frequencies for } \frac{1}{M \cdot \Delta t}\right)$$
where $\Delta t = 1/f_s$.

In [ ]:
import numpy
import scipy.signal as sig 
import matplotlib.pyplot as plt
import os
from tqdm import tqdm


# Cargar el archivo como enteros con signo de 8 bits
def load_cs8(filename):
    data = np.fromfile(filename, dtype=np.int8)
    I = data[0::2]  # Muestras pares como parte real
    Q = data[1::2]  # Muestras impares como parte imaginaria
    señal_compleja = I + 1j * Q
    return señal_compleja, I, Q

def welch_psd_full(signal, fs=1.0, segment_length=256, overlap=0.5, window_type='hamming'):
    signal = np.asarray(signal)
    N = len(signal)
    step = int(segment_length * (1 - overlap))

    # Selección y creación de la ventana
    if window_type == 'hamming':
        window = np.hamming(segment_length)
    elif window_type == 'hann':
        window = np.hanning(segment_length)
    elif window_type == 'rectangular':
        window = np.ones(segment_length)
    else:
        raise ValueError("Tipo de ventana no soportado")

    U = np.sum(window ** 2)
    K = (N - segment_length) // step + 1
    P_welch = np.zeros(segment_length)

    # Barra de progreso
    for k in tqdm(range(K), desc="Calculando Welch PSD"):
        start = k * step
        segment = signal[start:start + segment_length] * window

        # Calcular la FFT completa del segmento y obtener PSD
        X_k = np.fft.fft(segment)
        P_k = (1 / (fs * U)) * np.abs(X_k) ** 2

        P_welch += P_k

    P_welch /= K  # Promediar sobre los segmentos
    f = np.fft.fftfreq(segment_length, d=1.0 / fs)

    return f, P_welch


# raw_sig, I, Q = cargar_cs8(os.path.join(base_path,str(0)))
# Estimar la PSD usando la función ajustada de Welch con eliminación de la media

def exec_psd_full(x, fs, segment_length, overlap=0.5, window_type='hammming'):
    f, Pxx = welch_psd_full(x, fs, segment_length, overlap, window_type)

    return np.fft.fftshift(f), np.fft.fftshift(Pxx)